In [1]:
import pandas as pd
db = pd.read_csv('./dataset/ckextended.csv', delimiter=',')
db['Usage'].value_counts()

Training       734
PrivateTest     95
PublicTest      91
Name: Usage, dtype: int64

In [2]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  920 non-null    int64 
 1   pixels   920 non-null    object
 2   Usage    920 non-null    object
dtypes: int64(1), object(2)
memory usage: 21.7+ KB


In [3]:
db_train = db[db['Usage'] == 'Training']
db_train.drop('Usage', axis=1)

db_validation = db[db['Usage'] == 'PrivateTest']
db_validation.drop('Usage', axis=1)

db_test = db[db['Usage'] == 'PublicTest']
db_test.drop('Usage', axis=1)

,emotion,pixels
474,6,11 7 6 2 7 7 0 0 0 6 20 45 62 91 117 134 148 1...
475,6,59 66 63 64 64 65 36 11 15 15 12 20 35 49 88 1...
476,6,52 53 55 47 31 23 22 26 14 20 18 25 24 41 83 1...
477,6,77 76 44 28 17 14 8 9 7 7 10 12 18 8 12 41 52 ...
478,6,153 156 114 114 111 81 110 148 137 109 145 165...
...,...,...
906,5,43 39 41 42 14 13 10 15 21 23 18 18 17 14 15 1...
907,5,201 198 194 172 59 61 74 52 73 107 67 67 102 1...
908,5,69 71 74 75 24 1 0 2 6 6 3 1 3 5 24 16 12 17 2...
909,5,79 81 82 82 84 86 85 80 80 40 27 26 40 61 69 6...


In [4]:
from PIL import Image
import numpy as np
listona = list(map(int, db_train['pixels'][0].split(' ')))


In [5]:
X_train = []
y_train = [label for label in db_train['emotion']]

for img in db_train['pixels']:
    tmp = []
    for pixel in img.split(' '):
        tmp.append(int(pixel))
    X_train.append(tmp)

In [6]:
print(len(X_train), len(y_train))

734 734


In [7]:
import torch
X_train = torch.tensor(X_train, dtype=int)
y_train = torch.tensor(y_train, dtype=int)

In [5]:
from VGG import VGG19
import torch
import torch.nn as nn
import torch.optim as optim
net = VGG19()

if torch.cuda.is_available():
    net.cuda()
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)


In [6]:
from ck import CK

train = CK('Training')
trainloader = torch.utils.data.DataLoader(train, batch_size=128, shuffle=True, num_workers=1)

test = CK('Testing')
testloader = torch.utils.data.DataLoader(test, batch_size=5, shuffle=False, num_workers=1)


In [7]:

def train(epoch):
    print("Epoca:" + str(epoch))
    
    net.train()
    
    for inputs, labels in enumerate(trainloader):
        
        inputs, labels = inputs.cuda(), labels.cuda()
        
    
        # Azzeramento dei gradienti
        optimizer.zero_grad()
        
        # Forward pass
        outputs = net(inputs)
        
        # Calcolo della loss
        loss = criterion(outputs, labels)
        
        # Calcolo dei gradienti
        loss.backward()
        
        # Aggiornamento dei parametri
        optimizer.step()
         
    """with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in validation_dataloader:
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        accuracy = correct / total"""
        
    #print(f"Loss{loss.item()} , Accuracy{accuracy}")
    
def test(epoch):
    pass



In [8]:
for epoch in range(0, 5):
    train(epoch)
    test(epoch)

Epoca:0


AttributeError: 'int' object has no attribute 'cuda'